PART 1: Moving data into a dataframe and manipulating it

First, import libraries that we will use in this notebook

In [1]:
import numpy as np
import pandas as pd

In [2]:
from matplotlib import pyplot
from pandas import DataFrame

Now, we want to mount Google Drive to Colab to be able to read data from the CSV file

In [3]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


Use unix ls command to make sure that the file we will load is on MyDrive

In [ ]:
!ls drive/MyDrive/CstatWeatherNov19.csv

Insert a code cell below this one that contains code to import the csv data from the file into a dataframe df using Pandas read_csv  -- note that if you use interational charactersets, sometimes these can fail during read_csv().  Often including encoding = 'unicode_escape' in the read_csv() fixes this problem.

Use info() to look at the number and types of data that we loaded into the dataframe. Here we use the "dot notation." Dot notation allows us to refer to method of the object (method is a function exclusive to a specific class/object/instance) or attribute of the instance/object.  Methods have () at the end of the name, attributes do not.  So here's the info() method for the dataframe df:

In [ ]:
df.info()

Python machine learning tools that we will use extensively operate on numeric data.  We have Day which is an object type and Precip inch (rainfall data) is currently an object type.  


Let's look at the first few rows of the dataframe:

In [ ]:
df.head()

First, convert Day into a special type which will allow us to do date/time manipulations using Pandas to_datefime()

In [ ]:
df['Day'] = pd.to_datetime(df['Day'], format = '%m/%d/%Y')


Now insert a cell to check the info() again

Next, Precip inch.  Look at the values for precipitation to see why they are objects.

In [ ]:
df['Precip inch']

The use of T for trace rainfall is causing the problem. Trace means that detectible precipitation of less than 0.005 inches were detected.  For simplicity, substitute a small number for T. Note: trace in snow depth is defined as less than 0.05 inches

In [ ]:
df.loc[df['Precip inch']=='T', ['Precip inch']] = 0.005

Now, we've eliminated the Ts, but that doesn't automatically change the Dtype.  Change the Dtype to float

In [ ]:
df['Precip inch'] = df['Precip inch'].astype(float, errors = 'raise')

Again, insert a cell and use info() to check the type is now numeric

So, look at all the cleaned data. Since this is a small dataframe we can look at it all at once.  For large datasets, use df.head() to show the first 5 rows. df.tail(7) will show the last 7 rows in the dataframe.  Experiement with these in the next few cells.  Try the entire dataframe, then the first 5 rows and the last 7 rows

In [ ]:
#df
#df.head()
df.tail(7)

This dataframe is quite clean.  We could do some additional things like fixing consistency of capitalization of the feature names. df.rename() would change the column names.

Instead, lets work on a dirty data example.

This file DataClean.csv is on canvas. Insert some cells that: Go get the file, make sure it is in your MyDrive directory, and read it into a dataframe named dirtydf (following what we did above for the  weather data.)

In [ ]:
#see what it looks like
dirtydf.info()

This dataframe is a MESS.  Each feature (column) should have 9 observations or instances (rows). Let's take a look

In [ ]:
dirtydf

We can use the dataframe count method to see how many values we have in our datrframe.  NaN values are not counted in count(). Many are obviously missing.

In [ ]:
dirtydf.count()

First, let's fix the Date object like we did before. Insert cell below and run the cell.

In [ ]:
dirtydf['Date'] = pd.to_datetime(dirtydf['Date'], format = '%m/%d/%Y')
dirtydf

Since we have only one reading on Gage 4, doing statistical analysis will not be very useful for this column.  Let's just drop this entire column from our dataframe:

In [ ]:
dirtydf = dirtydf.drop(['Gage 4'], axis=1) #many ways of specifying how to drop this column. Check out pandas documentation for others

How does it look now

In [ ]:
dirtydf

Let's next drop the row from the day that Dr Lowe was sick and didn't take any readings

In [ ]:
dirtydf.drop([4], inplace=True) # drops index 4 (row 4) within same dataframe

In [ ]:
dirtydf

Now, let's fix those NaNs. Pandas dropna() and fillna() are very flexible. We can force the NA to a value (eg 0) or the min/max/mean/etc of that column. We can also replace it with the last valid entry. Let's try that:



In [ ]:
dirtydf['Gage 2'].fillna( method ='ffill', inplace = True) # use forward fill method
dirtydf

Lets look at info() again

In [ ]:
dirtydf.info()

We want all our fields to be numeric so that we can generate statistics.  Fix the Dtype of Gage 1:

In [ ]:
dirtydf['Gage 1'] = pd.to_numeric(dirtydf['Gage 1'])
dirtydf

For reasons that will become clearer when we do ML classification, we want to have the Measurer be categorical Dtype rather than Object. We can accomplish this with the following:

In [ ]:
dirtydf["Measurer"] = dirtydf["Measurer"].astype("category")

Let's take another look at the dataframe

In [ ]:
dirtydf.info()
dirtydf

Later, we may turn the categorical feature into a numeric value.

Now, what are we going to do about the obviously erroneous reading of Gage 2 on Oct 6?

It is very obvious this is a outlier due to some error.Now let's look at that entry for Gage 2 which seems to be an error.

We will generally just use visualization techniques for this class to help us recognize outliers.  This example will be very obvious, but let's go through a set of techniques that can help with datasets with many more observations and with outliers which are not so obvious.  

First, look at a sort of the data values for the Gage 2 feature:

In [ ]:
dirtydf['Gage 2'].sort_values()

Note if there were hundreds or thousands of observations, we'd need to look at head() or tail().  Practice here by inserting a cell to print the first 3 and the last 3 entries of the sorted data

Sorted features make it easier to see what may be abnormally small or large.  Another visualization technique is boxplot. Boxplots show the distribution of the data for a row or column with a box for the data from the 1st to the 3rd quartile and "wiskers" for data 1.5X the extent of the box.  For many features, data values outside the wiskers can be considered outliers. Be careful, however, with features where the data is exponential in scale. But our values are not exponential, so let's use box plot "wiskers" to guide us.  Let's start by generating a boxplot for Gage 3, which seems to have no outliers:

In [ ]:
boxplot = dirtydf.boxplot(column=['Gage 3'])

Now repeat this for Gage 2: Insert a cell to create a boxplot for Gage 2

Now this value is obviously an error. The outlier is way outside the box and wiskers, which are compressed into a single green line in this plot.  

We can (1) try to fix the error if we can determine what a likely correct value is, (2) drop the feature or the observation using dirtydf.drop(), or (3) we can replace outliers with another value 0, min, max, mean, ...

By examination it seems like two measurements, each without their decimal point were incorrectly entered for this datum.  Let's assume that the first four digits with the decimal point after the first digit is the correct reading for Gage 2 on Oct 6.

We could try to fix that by doing a replacement: dirtydf.at[index, column] = dirtydf.at[index, column] / 1000000, with the correct index and column label

However, that leaves the erroneous 4 digits at the end of the number.  An alternative would be to first do modular division on the incorrect datum to clear the last 4 digits followed by a real division to adjust the decimal point.

Insert cells to correct the datum at index 5 of feature Gage 2 as described.

Show your fixed dataframe:

In [ ]:
dirtydf

OK, so now that it is clean let's do some data frame manpulations:
Let's add an additional column (attribute + data = feature) that is the average of the readings of Gage 1 and Gage 2.  We will use a new series to compute the average then we will insert thate as a new column in the dataframe:

In [ ]:
avgG1G2=dirtydf[["Gage 1", "Gage 2"]].mean(axis=1)

In [ ]:
avgG1G2

We can use several techniques to insert this numpy series back into our dataframe df.insert(position, name, series to insert) works well for this.  df.assign() will also work.  Open a cell and use Pandas dataframe .insert() to add avgG1G2 into the dataframe dirtydf at position 4.

now show the dataframe to verify that the new feature has been added to the dataframe.

In [ ]:
dirtydf

Let's look at subsets of our dataset.  Let's just look at the data Measured by Abe:

In [ ]:
dirtydf['Measurer']=='Abe'# which indices correspond to Abe's measurements


In [ ]:
dirtydf[dirtydf['Measurer']=='Abe']# subset of dataset from Abe's measurements


In [ ]:
dirtydf.loc[dirtydf['Measurer']=='Abe',['Gage 3']]# the readings from Gage 3 that were measured by Abe

Being able to subset our observations by a value (or range of values) in one feature is important.  Equally important is being able to subset the corresponding data from another feature for the subset determined by another feature.

Remember back to our eliminating the observation that had missing data since Dr. Lowe was out sick.  In that case we used index 4 to eliminate that observation with df.drop().
If Dr. Lowe turned out to consistently be a bad Gage reader, we could use the above subsetting technique to select only observations made by other observers.

OK, so now back to our weather data in the dataframe df:

In [ ]:
df.count()

Count includes numeric values (not a number entries or NANs are not counted)

Compute the Proportion of observations with precipitation.

In [ ]:
((df['Precip inch'] != 0).sum()/df['Precip inch'].count())

We could do other, more complex, Proportion calculations. Proportion of days with more than a Trace of rain.  For Percent, add a multiplication by 100.

Histograms are a good way to look at Frequency Distribution.  
First, use Matplotlib to create a simple histogram

In [ ]:

df.hist(column='high degree F')

Adding axis labels, title, gridlines, limits, etc using Matplotlib a better looking histogram results

In [ ]:
# An "interface" to matplotlib.axes.Axes.hist() method
n, bins, patches = pyplot.hist(x=df['high degree F'], bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
pyplot.grid(axis='y', alpha=0.75)
pyplot.xlabel('high temp (F)')
pyplot.ylabel('Frequency')
pyplot.title('Nov 2019 College Station daily high temperature')
maxfreq = n.max()
# Set a clean upper y-axis limit.
pyplot.ylim(ymax=np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10)

If we want more bins for the histogram adjust the number of bins (6 to 16 is considered best practice). Insert the code after this cell to creat a 16 bin histogram of the daily high temperature:

Measures of Central Tendancy

Mean, median, mode can all be computed on the dataframe columns. Pandas has a method for mean(), median(), and mode() for datframes.  Let's find the means of the daily high temperature:

In [ ]:
print("mean:", df['high degree F'].mean())

OR we could find the mean of all features:

In [ ]:
df.mean()

Insert cells to find the median of the high temperature and the mode of the high temperature.

Now insert a cell to show the medians of all the features in df

For Geometric means use the stat library in scipy (note: geometric mean is not suited for mean temperature)


In [ ]:
from scipy.stats.mstats import gmean
gmean(df['high degree F'])

Likewise harmonic mean (also not relevant)

In [ ]:
from scipy.stats.mstats import hmean
hmean(df['high degree F'])

Moving on to Measures of Dispersion or Variation

First, for range we can use numpy "peak to peak" function ptp() to compute ranges. Alternatively, we could in Pandas use df.max() and df.min() and a little math would give the same result,

In [ ]:
np.ptp(df['high degree F'])

Next use df.var() to compute variance.  You adjust variance for population vs sample by using the ddof (delta degrees of freedom) parameter.

In [ ]:
# sample variance
df['high degree F'].var()

In [ ]:
#population variance
df['high degree F'].var(ddof=0)

For standard deviation, the code is similar

In [ ]:
# sample standard deviation
df['high degree F'].std()

In [ ]:
#popuation standard variation
df['high degree F'].std(ddof=0)

Ranking.  Percent rank uses df.rank(). First lets see the high temperatures for the month

In [ ]:
df['high degree F']

Now for each of those readings, rank shows the percentile rank of that value.

In [ ]:
df['high degree F'].rank(pct=True)

Now quantile rank to get data quartiles.

In [ ]:
pd.qcut(df['low degree F'], q=4)


Finally, a summary of the characteristics of the dataframe which includes many of the statistics can be displayed by using describe()

In [ ]:
df.describe()

More sophisticated distribution analysis and ploting can be done with the Dataframe data and Matplotlib. For example to compare high and low temperature distributionshistograms (and Gaussian Kernel Density Estimates as distribution model) we can use this plot:

In [ ]:

fig, ax = pyplot.subplots()
df[["high degree F", "low degree F"]].plot.kde(ax=ax, legend=False, title='Nov 2019 College Station Daily Temperature')
df[["high degree F", "low degree F"]].plot.hist(density=True, ax=ax, alpha=0.3)
ax.set_ylabel('Probability')
ax.grid(axis='y')
ax.set_facecolor('#d8dcd6')






PART 2: Loading and Cleaning Leaf Blower Data

In [ ]:
# change the directory as needed
!ls drive/MyDrive/ECEN250_LeafBlowers.csv

In [ ]:
# importing dataset
df = pd.read_csv('drive/MyDrive/ECEN250_Lab2_LeafBlowers.csv')


In [ ]:
df.info()

Let's look at what we read in to make sure it's what we expected:

In [ ]:
df.head(10)

Manufacturer, model, retail, and source are going to be non-numeric by nature. These are currently objects, because some entries may be numeric and some string. We will leave them as they are since they will not be used in our statistics or ML data analysis. The feature motor type contains strings: brushed, brushless, or unspecified. It is currently an object datatype. This feature -- is categorical.  Is it nominal or ordinal?? We will use it a lot in
our analysis so let's turn it into a numerical that we can manipulate. We can do that with the following python cell (notice when we use flag inplace=True we are directly modifying our df -- if not done inplace replace() returns a new modified df):

In [ ]:
df['motor type'].replace(['brushless', 'unspecified', 'brushed'],
                        [0, 0.5, 1], inplace=True)

Let's verify this worked by doing df.head(10) again

In [ ]:
df.head(10)

To make sure it it indeed now numeric, let's do df.info() again:

In [ ]:
df.info()

Good, so now everything we will use today is numeric!

If for some reason this didn't work -- or you decided that you want to just make them categorical for now and numeric later, go back to the cell that imports the csv file.  Re-execute that cell and all the cells up to the one you are changing.  THIS IS WHY WE DON'T MODIFY OUR CSV SOURCE FILE!

Take another look at the results of the df.info() above -- the non-null counts will differ if you have some csv rows where you have not filled in each field.  In the dataframe those entries will be NaN. Our statistics and modeling will be messed up by those NaN values. If you do not have 50 entries in which all
features have a value, you need to stop now and find a few more to add to your csv.  Then restart the runtime and rerun your notebook to here.

If you have NaNs in your dataframe you can use the techniques you learned earlier to fix them here:

Use df.info() to verify no remaining NaNs

In [ ]:
df.info()

We are almost done --- we first should write out the data from our dataframe since we will use this cleaned data for Lab 3.  To do that you can write out your data into a CSV.  The flag index=False prevents the Index values (column 0) from being written to the csv.  We do this because read_csv by default assigns index values as the data is being read from the csv file.

In [ ]:
df.to_csv('drive/MyDrive/ECEN250_Lab2_LeafBlowersClean.csv', index=False)

Now make sure that this notebook shows all your executed cells.  If it does not, you can restart the runtime and runall or you can restart the runtime and individually reexecute all of your cells.

Now download your notebook as both an ipynb file (use a name that includes your name please) and a PDF file (to create the PDF in Colab, go to File > Print and select Save as PDF), and upload both files to Canvas for your submission for Lab 2.